### Ingestion de la carpeta "production_country"


In [0]:
dbutils.widgets.text("p_environment", "")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/common_functions"

In [0]:
%run "../includes/configuration"

#### Paso 1 - Leer carpeta con JSON usando "DataFrameReader" de spark

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
productions_countries_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("countryId", IntegerType(), True)
])

In [0]:
prodtuctions_countries_df = spark.read \
                .schema(productions_countries_schema) \
                .option("multiLine", True) \
                .json(f"{bronze_folder_path}/{v_file_date}/production_country")

In [0]:
display(prodtuctions_countries_df)

#### Paso 2 - Renombrar las columnas y añadir nuevas
1. "movieId" renombrar a "movie_id"
2. "countryId" renombrar a "country_id"
3. agregar las columnas "ingestion_date" y "environment"

In [0]:
productions_countries_final_df = add_ingestion_date(prodtuctions_countries_df) \
                                .withColumnsRenamed({"movieId": "movie_id",
                                              "countryId": "country_id"}) \
                                .withColumn("environment", lit(v_environment)) \
                                .withColumn("file_date", lit(v_file_date))

In [0]:
display(productions_countries_final_df)

#### Paso 3 - Eliminar columnas no deseadas

In [0]:
#movies_languages_final_df = movies_languages_with_columns_df.drop(col("languageRoleId"))

In [0]:
#display(movies_languages_final_df)

#### Paso 4 - Escribir en parquet

In [0]:
#overwrite_partition(productions_countries_final_df, "movie_silver", "productions_countries", "file_date")

In [0]:
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.country_id = src.country_id AND tgt.file_date = src.file_date'
merge_delte_lake(productions_countries_final_df, "movie_silver", "productions_countries", silver_folder_path, merge_condition, "file_date")

In [0]:
%sql
SELECT file_date, count(1)
FROM movie_silver.productions_countries
GROUP BY file_date;

In [0]:
dbutils.notebook.exit("success")